In [1]:
import sys
print(sys.executable)




C:\Users\Johanna\anaconda3\envs\AI_project\python.exe


In [40]:
!pip install scikeras

In [52]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from xgboost import XGBRegressor
from tensorflow.keras.layers import  Dropout
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from scikeras.wrappers import KerasRegressor
from tensorflow.keras.optimizers import Adam
from keras.models import load_model
from tensorflow.keras.layers import Input


In [4]:
df_lagged_all = pd.read_parquet("data/processed/df_lagged_all.parquet")
df_2_full = pd.read_parquet("data/processed/df_2_full_v2.parquet")
df_all=pd.read_parquet("data/processed/df_all.parquet")



In [5]:
target_sensor = 1076
same_portal="55620"
neighbour_portal = "56160"

same_portal_sensors = df_2_full[df_2_full['PORTAL_clean'] == same_portal]['DP_ID'].unique()
same_sensors = [s for s in same_portal_sensors if s != target_sensor]


neighbour_sensors = df_2_full[df_2_full['PORTAL_clean'] == neighbour_portal]['DP_ID'].unique()
all_sensors=df_2_full['DP_ID'].unique()
except_target_sensors = [s for s in all_sensors if s != target_sensor]




In [6]:


flow_features_same = [
    f'SENSOR_{sensor}_FLOW_lag_{i+1}'
    for sensor in same_sensors
    for i in range(15)
]
flow_features_neighbour = [
    f'SENSOR_{sensor}_FLOW_lag_{i+1}'
    for sensor in neighbour_sensors
    for i in range(15)
]
speed_features_same = [
    f'SENSOR_{sensor}_SPEED_lag_{i+1}'
    for sensor in same_sensors
    for i in range(15)
]
speed_features_neighbour = [
    f'SENSOR_{sensor}_SPEED_lag_{i+1}'
    for sensor in neighbour_sensors
    for i in range(15)
]        

xgb-boost : hyperparameter tuning

In [10]:
df_train, df_test = train_test_split(df_lagged_all, test_size=0.2, random_state=42)

#split_time = df_2_full['Datetime'].quantile(0.8)
#print(split_time)

#df_train = df_lagged_all[df_lagged_all['Datetime'] <= split_time]
#df_test = df_lagged_all[df_lagged_all['Datetime'] > split_time]


In [ ]:
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.001,0.005,0.01, 0.05],
    "subsample": [0.6, 0.8, 1.0],
    "colsample_bytree": [0.6, 0.8, 1.0]
}


In [18]:


random_search_flow = RandomizedSearchCV(
    estimator=XGBRegressor(random_state=42),
    param_distributions=param_grid,
    n_iter=30,  # Anzahl zufälliger Kombinationen
    scoring="neg_root_mean_squared_error",
    cv=3,
    verbose=2,
    n_jobs=-2
)

random_search_flow.fit(df_train[flow_features_same], df_train["FLOW_future_sum"])
print("Beste Parameter (FLOW):", random_search_flow.best_params_)
model_flow = random_search_flow.best_estimator_
y_pred = model_flow.predict(df_test[flow_features_same])
y_test = df_test['FLOW_future_sum']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"XGBoost Regression (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")


random_search_speed = RandomizedSearchCV(
    estimator=XGBRegressor(random_state=42),
    param_distributions=param_grid,
    n_iter=30,  # Anzahl zufälliger Kombinationen
    scoring="neg_root_mean_squared_error",
    cv=3,
    verbose=2,
    n_jobs=-2
)


random_search_speed.fit(df_train[speed_features_same], df_train["SPEED_future_mean"])
print("Beste Parameter (FLOW):", random_search_speed.best_params_)
model_speed = random_search_speed.best_estimator_

y_pred = model_speed.predict(df_test[speed_features_same])
y_test = df_test['SPEED_future_mean']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"XGBoost Regression (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Beste Parameter (FLOW): {'subsample': 0.8, 'n_estimators': 100, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.8}
XGBoost Regression (FLOW) -> RMSE: 28.452, MAE: 20.456, R²: 0.882
Fitting 3 folds for each of 30 candidates, totalling 90 fits


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Beste Parameter (FLOW): {'subsample': 0.8, 'n_estimators': 100, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 1.0}
XGBoost Regression (SPEED) -> RMSE: 0.813, MAE: 0.398, R²: 0.740


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [19]:
#neighbour portal

random_search_flow = RandomizedSearchCV(
    estimator=XGBRegressor(random_state=42),
    param_distributions=param_grid,
    n_iter=30,  # Anzahl zufälliger Kombinationen
    scoring="neg_root_mean_squared_error",
    cv=3,
    verbose=2,
    n_jobs=-2
)

random_search_flow.fit(df_train[flow_features_neighbour], df_train["FLOW_future_sum"])
print("Beste Parameter (FLOW):", random_search_flow.best_params_)
model_flow = random_search_flow.best_estimator_
y_pred = model_flow.predict(df_test[flow_features_neighbour])
y_test = df_test['FLOW_future_sum']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"XGBoost Regression (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")


random_search_speed = RandomizedSearchCV(
    estimator=XGBRegressor(random_state=42),
    param_distributions=param_grid,
    n_iter=30,  # Anzahl zufälliger Kombinationen
    scoring="neg_root_mean_squared_error",
    cv=3,
    verbose=2,
    n_jobs=-2
)


random_search_speed.fit(df_train[speed_features_neighbour], df_train["SPEED_future_mean"])
print("Beste Parameter (FLOW):", random_search_speed.best_params_)
model_speed = random_search_speed.best_estimator_

y_pred = model_speed.predict(df_test[speed_features_neighbour])
y_test = df_test['SPEED_future_mean']

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"XGBoost Regression (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")

Fitting 3 folds for each of 30 candidates, totalling 90 fits
Beste Parameter (FLOW): {'subsample': 0.8, 'n_estimators': 200, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
XGBoost Regression (FLOW) -> RMSE: 24.096, MAE: 16.871, R²: 0.915
Fitting 3 folds for each of 30 candidates, totalling 90 fits


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Beste Parameter (FLOW): {'subsample': 0.6, 'n_estimators': 100, 'max_depth': 7, 'learning_rate': 0.05, 'colsample_bytree': 0.6}
XGBoost Regression (SPEED) -> RMSE: 0.934, MAE: 0.421, R²: 0.657


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Feedforward Neural Network (NN)

In [42]:
 def construct_network_model(layer_sizes,dropout_rate,input_dim,learning_rate):
  # Create a Sequential model, which is a linear stack of layers.
  model = Sequential()
  model.add(Input(shape=(input_dim,)))
  for size in layer_sizes:
        model.add(Dense(size, activation='relu'))
        if dropout_rate > 0:
            model.add(Dropout(dropout_rate))
  model.add(Dense(1))  # Regression
  optimizer = Adam(learning_rate=learning_rate)
  model.compile(optimizer=optimizer, loss='mae', metrics=['mae'])
  return model

In [43]:
early_stop = EarlyStopping(monitor='val_mae', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_mae', factor=0.5, patience=3)


In [44]:
param_dist = {
    "model__learning_rate": [0.0005, 0.001, 0.005],
    "batch_size": [32, 64],
    "epochs": [200],
    "model__layer_sizes": [
        [128, 64, 32],
        [64, 32],
        [64, 64],
        [256, 128, 64],
        [32, 32, 32],
        [128, 128]
    ],
    "model__dropout_rate": [0.0, 0.2, 0.3, 0.5]
}



In [55]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(df_train[flow_features_same])
X_test_scaled  = scaler.transform(df_test[flow_features_same])
y_train = df_train["FLOW_future_sum"]
y_test = df_test["FLOW_future_sum"]



model_flow = KerasRegressor(
    model=construct_network_model,
    input_dim=X_train_scaled.shape[1],
    verbose=0
)

#checkpoint = ModelCheckpoint("model/randomsearch_same_flow.keras", monitor='val_mae', verbose=1, save_best_only=True, mode='min')
callbacks = [early_stop, reduce_lr]#, checkpoint]

random_search_flow = RandomizedSearchCV(
    estimator=model_flow,
    param_distributions=param_dist,
    n_iter=10,  
    cv=2,
    scoring='neg_mean_absolute_error',
    verbose=2,
    n_jobs=1
)

random_search_flow.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    callbacks=[early_stop]
)

# Bestes Modell speichern
random_search_flow.best_estimator_.model_.save("model/randomsearch_same_flow.keras")

Fitting 2 folds for each of 10 candidates, totalling 20 fits
[CV] END batch_size=32, epochs=200, model__dropout_rate=0.5, model__layer_sizes=[128, 128], model__learning_rate=0.0005; total time= 2.8min
[CV] END batch_size=32, epochs=200, model__dropout_rate=0.5, model__layer_sizes=[128, 128], model__learning_rate=0.0005; total time= 2.8min
[CV] END batch_size=64, epochs=200, model__dropout_rate=0.5, model__layer_sizes=[128, 64, 32], model__learning_rate=0.001; total time= 1.6min
[CV] END batch_size=64, epochs=200, model__dropout_rate=0.5, model__layer_sizes=[128, 64, 32], model__learning_rate=0.001; total time= 1.7min
[CV] END batch_size=32, epochs=200, model__dropout_rate=0.2, model__layer_sizes=[256, 128, 64], model__learning_rate=0.005; total time= 3.3min
[CV] END batch_size=32, epochs=200, model__dropout_rate=0.2, model__layer_sizes=[256, 128, 64], model__learning_rate=0.005; total time= 3.2min
[CV] END batch_size=32, epochs=200, model__dropout_rate=0.0, model__layer_sizes=[32, 32, 

In [56]:
y_pred=random_search_flow.best_estimator_.model_.predict(X_test_scaled)
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"Fast forward NN (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")


457/457 ━━━━━━━━━━━━━━━━━━━━ 0s 614us/step
Fast forward NN (FLOW) -> RMSE: 29.521, MAE: 20.533, R²: 0.873


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(df_train[speed_features_same])
X_test_scaled  = scaler.transform(df_test[speed_features_same])
y_train = df_train["SPEED_future_mean"]
y_test = df_test["SPEED_future_mean"]



model_speed = KerasRegressor(
    model=construct_network_model,
    input_dim=X_train_scaled.shape[1],
    verbose=0
)

#checkpoint = ModelCheckpoint("model/randomsearch_same_flow.keras", monitor='val_mae', verbose=1, save_best_only=True, mode='min')
callbacks = [early_stop, reduce_lr]#, checkpoint]

random_search_speed = RandomizedSearchCV(
    estimator=model_speed,
    param_distributions=param_dist,
    n_iter=10,  
    scoring='neg_mean_absolute_error',
    verbose=2,
    n_jobs=4
)

random_search_speed.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    callbacks=[early_stop]
)

# Bestes Modell speichern
random_search_speed.best_estimator_.model_.save("model/randomsearch_same_speed.keras")

In [33]:

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(df_train[flow_features_same])
X_test_scaled  = scaler.transform(df_test[flow_features_same])


model_flow= Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),  # Eingabeschicht
    Dense(32, activation='relu'),                                           # Versteckte Schicht
    Dense(16, activation='relu'),                                           # Weitere versteckte Schicht
    Dense(1)                                                                 # Ausgangsschicht für Regression
])

model_flow.compile(optimizer='adam', loss='mse', metrics=['mae'])


history_flow = model_flow.fit(
    X_train_scaled, df_train["FLOW_future_sum"],
    validation_split=0.2,  # 20% der Trainingsdaten für Validierung
    epochs=50,
    batch_size=32,
    verbose=0
)



y_pred = model_flow.predict(X_test_scaled)
y_test=df_test["FLOW_future_sum"]

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"Fast forward NN (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(df_train[speed_features_same])
X_test_scaled  = scaler.transform(df_test[speed_features_same])

model_speed = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),  # Eingabeschicht
    Dense(32, activation='relu'),                                           # Versteckte Schicht
    Dense(16, activation='relu'),                                           # Weitere versteckte Schicht
    Dense(1)                                                                 # Ausgangsschicht für Regression
])

model_speed.compile(optimizer='adam', loss='mse', metrics=['mae'])


history_speed = model_speed.fit(
    X_train_scaled, df_train["SPEED_future_mean"],
    validation_split=0.2,  # 20% der Trainingsdaten für Validierung
    epochs=50,
    batch_size=32,
    verbose=0
)


y_pred = model_speed.predict(X_test_scaled)
y_test=df_test["SPEED_future_mean"]
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"Fast forward NN (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")



C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


457/457 ━━━━━━━━━━━━━━━━━━━━ 0s 553us/step
Fast forward NN (FLOW) -> RMSE: 29.420, MAE: 20.936, R²: 0.874


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


457/457 ━━━━━━━━━━━━━━━━━━━━ 0s 549us/step
Fast forward NN (SPEED) -> RMSE: 0.806, MAE: 0.411, R²: 0.744


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [32]:

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(df_train[flow_features_neighbour])
X_test_scaled  = scaler.transform(df_test[flow_features_neighbour])


model_flow= Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),  # Eingabeschicht
    Dense(32, activation='relu'),                                           # Versteckte Schicht
    Dense(16, activation='relu'),                                           # Weitere versteckte Schicht
    Dense(1)                                                                 # Ausgangsschicht für Regression
])

model_flow.compile(optimizer='adam', loss='mse', metrics=['mae'])


history_flow = model_flow.fit(
    X_train_scaled, df_train["FLOW_future_sum"],
    validation_split=0.2,  # 20% der Trainingsdaten für Validierung
    epochs=50,
    batch_size=32,
    verbose=0
)



y_pred = model_flow.predict(X_test_scaled)
y_test=df_test["FLOW_future_sum"]

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"Fast forward NN (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(df_train[speed_features_neighbour])
X_test_scaled  = scaler.transform(df_test[speed_features_neighbour])

model_speed = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),  # Eingabeschicht
    Dense(32, activation='relu'),                                           # Versteckte Schicht
    Dense(16, activation='relu'),                                           # Weitere versteckte Schicht
    Dense(1)                                                                 # Ausgangsschicht für Regression
])

model_speed.compile(optimizer='adam', loss='mse', metrics=['mae'])


history_speed = model_speed.fit(
    X_train_scaled, df_train["SPEED_future_mean"],
    validation_split=0.2,  # 20% der Trainingsdaten für Validierung
    epochs=50,
    batch_size=32,
    verbose=0
)


y_pred = model_speed.predict(X_test_scaled)
y_test=df_test["SPEED_future_mean"]
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae  = mean_absolute_error(y_test, y_pred)
r2   = r2_score(y_test, y_pred)

print(f"Fast forward NN (SPEED) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")



C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


457/457 ━━━━━━━━━━━━━━━━━━━━ 0s 555us/step
Fast forward NN (FLOW) -> RMSE: 25.049, MAE: 18.017, R²: 0.908


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


457/457 ━━━━━━━━━━━━━━━━━━━━ 0s 546us/step
Fast forward NN (SPEED) -> RMSE: 0.988, MAE: 0.472, R²: 0.616


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


LSTM

In [34]:
def create_sequences(X, y, seq_length=15, horizon=15):
    Xs, ys = [], []
    for i in range(len(X) - seq_length - horizon + 1):
        Xs.append(X[i:i+seq_length])
        ys.append(y[i+seq_length:i+seq_length+horizon])
    return np.array(Xs), np.array(ys)


In [35]:


X = df_lagged_all[flow_features_neighbour].values
y = df_lagged_all['FLOW_filled'].values


X_seq, y_seq = create_sequences(X, y, seq_length=15, horizon=15)


In [36]:
n_features = X_seq.shape[2]

model = Sequential([
    LSTM(64, activation='tanh', input_shape=(15, n_features)),
    Dense(15)  # 15 Output-Werte für 15-Minuten-Vorhersage
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [37]:
split = int(len(X_seq) * 0.8)
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1,verbose=1)


y_pred = model.predict(X_test)




Epoch 1/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 32.1948 - mae: 4.1489 - val_loss: 20.3755 - val_mae: 3.3618
Epoch 2/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 17.0597 - mae: 3.1468 - val_loss: 19.8722 - val_mae: 3.2647
Epoch 3/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 16.9467 - mae: 3.1308 - val_loss: 19.8680 - val_mae: 3.2506
Epoch 4/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 16.9074 - mae: 3.1260 - val_loss: 20.0148 - val_mae: 3.2685
Epoch 5/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 16.8527 - mae: 3.1213 - val_loss: 19.7370 - val_mae: 3.2497
Epoch 6/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 16.7988 - mae: 3.1159 - val_loss: 19.8592 - val_mae: 3.2885
Epoch 7/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 16.7502 - mae: 3.1085 - val_loss: 19.6280 - val_mae: 3.2491
Epoch 8/20
1642/1642 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 16.7225 - mae: 3.1058 - val_loss: 19.9907 - val_mae: 3.2482
Epoch 9/20
1642/1642 ━━━

In [38]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"LSTM (FLOW) -> RMSE: {rmse:.3f}, MAE: {mae:.3f}, R2: {r2:.3f}")

y_test_sum = y_test.sum(axis=1)
y_pred_sum = y_pred.sum(axis=1)

# Metriken berechnen
rmse = mean_squared_error(y_test_sum, y_pred_sum, squared=False)
mae = mean_absolute_error(y_test_sum, y_pred_sum)
r2 = r2_score(y_test_sum, y_pred_sum)

print(f"LSTM Sum Forecast (FLOW) → RMSE: {rmse:.3f}, MAE: {mae:.3f}, R²: {r2:.3f}")

LSTM (FLOW) -> RMSE: 3.994, MAE: 2.911, R2: 0.633
LSTM Sum Forecast (FLOW) → RMSE: 36.944, MAE: 22.799, R²: 0.820


C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Johanna\anaconda3\envs\AI_project\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
